### 二次元リストから生成

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

import pandas as pd

# 1 各行を一つのリストとして並べる

list = [
    [1, 100, 0.33, 'AAA', 'AAA100'], 
    [2, 200, 0.67, 'BBB', 'BBB200'], 
    [3, 300, 1, 'CCC', 'CCC300'], 
    [4, 400, 1.33, 'DDD', 'DDD400'], 
    [5, 500, 1.67, 'EEE', 'EEE500'], 
    [6, 600, 2, 'FFF', 'FFF600']
]

# 2 表に変換
# pd.DataFrame(list) : pandasのDataFrameで表に変換
df = pd.DataFrame(list)
# 行名・列名は自動で割り振られた番号になっている

# 3 行名の変更
df.index = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']

# 4 列名の変更
df.columns = ['col0', 'col1', 'col2', 'col3', 'col4']

df

# 1 各行を一つのリストとして並べる
list = [
[1, 100, 0.33, 'AAA', 'AAA100'], 
[2, 200, 0.67, 'BBB', 'BBB200'], 
[3, 300, 1, 'CCC', 'CCC300'], 
[4, 400, 1.33, 'DDD', 'DDD400'], 
[5, 500, 1.67, 'EEE', 'EEE500'], 
[6, 600, 2, 'FFF', 'FFF600']]


#2 表に変換(オプションで行列名を指定)
ind = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']
col = ['col0', 'col1', 'col2', 'col3', 'col4']

df = pd.DataFrame(list, index=ind, columns=col)


df

# 1 各行を一つのリストにする
list = [
['row0', 1, 100, 0.33, 'AAA', 'AAA100'], 
['row1', 2, 200, 0.67, 'BBB', 'BBB200'], 
['row2', 3, 300, 1, 'CCC', 'CCC300'], 
['row3', 4, 400, 1.33, 'DDD', 'DDD400'], 
['row4', 5, 500, 1.67, 'EEE', 'EEE500'], 
['row5', 6, 600, 2, 'FFF', 'FFF600'], ]

# 2 表に変換
df = pd.DataFrame(list)

# 3 列名を変更
df.columns = ['', 'col0', 'col1', 'col2', 'col3', 'col4']

# 4 行名となる列を指定
df = df.set_index('')

df

# 1 各行ごとのリストを作成

listA = [1, 100, 0.33, 'AAA', 'AAA100']
listB = [2, 200, 0.67, 'BBB', 'BBB200']
listC = [3, 300, 1, 'CCC', 'CCC300']
listD = [4, 400, 1.33, 'DDD', 'DDD400']
listE = [5, 500, 1.67, 'EEE', 'EEE500']
listF = [6, 600, 2, 'FFF', 'FFF600']

# 2 まとめて表に変換

df = pd.DataFrame([listA, listB, listC, listD, listE, listF])

# 3 行名の変更
df.index = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']

# 4 列名の変更
df.columns = ['col0', 'col1', 'col2', 'col3', 'col4']

df

In [ ]:
cg_c_data = return_dict_summary_fixed()

In [ ]:
x = cg_c_data["processes"]
y = cg_c_data["ICNVRT"]

In [ ]:
cg_icnvrt_lr = ModelLog10(x, y, "CG", "ICNVRT")

In [ ]:
cg_icnvrt_lr.calc_lr()

In [ ]:
cg_icnvrt_lr.plot_graph()

In [ ]:
max_in_y = max(y)
# 最大値の個数を取得
y.tolist().count(max_in_y)
# 最大値のインデックスで最小のもの
y.tolist().index(max_in_y)

In [ ]:
class ModelBranch(ModelBase):
    
    def calc_lr(self):
        # 後述する t を算出するための処理
        max_in_train_y = max(self.train_y)
        max_in_train_y_first_index = self.train_y.tolist().index(max_in_train_y)
        # 分岐点のインデックスを t とする
        t = max_in_train_y_first_index
        self.t = t
        x_train_1 = self.train_x[:t]
        x_train_2 = self.train_x[t:]
        y_train_1 = self.train_y[:t]
        y_train_2 = self.train_y[t:]
        self.lr1 = LinearRegression()
        self.lr1.fit(x_train_1, y_train_1)
        self.lr2 = LinearRegression()
        self.lr2.fit(x_train_2, y_train_2)
        
    def calc_mape_score(self):
        x_test = self.test_x
        y_test = self.test_y
        y_test_predicted = self.lr2.predict(x_test)
        self.mape_score = float(mape_score(y_test, y_test_predicted))
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        
        # 回帰曲線を二つのモデルで分割するための処理
        x_model_line = self.x_model_line
        t_in_model_line = 0
        for i in range(len(x_model_line)):
            if (self.train_x[self.t] < x_model_line[i]):
                t_in_model_line = i
                break
            else:
                t_in_model_line = i
        
        x_model_line1 = self.x_model_line[:t_in_model_line]
        x_model_line2 = self.x_model_line[t_in_model_line:]
        y_model_line1 = self.lr1.predict(x_model_line1)
        y_model_line2 = self.lr2.predict(x_model_line2)

        plt.plot(x_model_line1, y_model_line1, color="red")
        plt.plot(x_model_line2, y_model_line2, color="red")
#         plt.plot(self.test_x, self.test_y, color="yellow")
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
        

In [ ]:
model_branch = ModelBranch(x, y, "CG", "ICNVRT")

model_branch.calc_lr()

model_branch.calc_mape_score()

print(model_branch.mape_score)

model_branch.plot_graph()

In [31]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [32]:
# ~~~_excludeBTSP, ~~~_onlyBTSP はそれぞれのベンチマークで取得したプロセス数
processes_excludeBTSP = [1, 2, 4, 8, 16, 32, 64, 128, 256]
processes_onlyBTSP = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 169, 196, 225]

In [33]:
# 引数に横軸：プロセス数orベンチマーククラス, 縦軸：関数名となっているデータフレームを取る
# 返値として
# rowData:プロセス数のリスト もしくは プロセス数のリスト (引数に由来)
# 各種関数名：実行回数のリスト
# 以上のような関係の辞書を返す

def return_dict_Data(DataFrame):
    columns = DataFrame.columns.to_numpy()
    index = DataFrame.index.to_numpy()
    # 返値となる辞書return_dictに引数のデータフレームの列名(プロセス数orベンチマーククラス)を格納
    return_dict = {"rowData":columns}
    for index_name in index:
        return_dict[index_name] = DataFrame.T[index_name].to_numpy()
    
    return return_dict


In [34]:
# fixed_class_list : return_fixed_class()の実行時に

fixed_class_list = [0] * len(benchmarks)
for i in range(len(fixed_class_list)):
    if ( benchmarks[i] == "bt" or benchmarks[i] == "sp" ):
        processes = processes_onlyBTSP
    else:
        processes = processes_excludeBTSP
    fixed_class_list[i] = return_fixed_class(BenchMark = benchmarks[i], Processes = processes, FixedBenchMarkClass = "C")

fixed_class_DataFrame = [0] * len(fixed_class_list)
for i in range(len(fixed_class_list)):
    fixed_class_DataFrame[i] = pd.concat(fixed_class_list[i] , axis=1)

print(benchmarks[0])

bt


In [35]:
print(type(fixed_class_DataFrame[0]))

dict_sample = return_dict_Data(fixed_class_DataFrame[0])

print(type(dict_sample))

dict_sample

<class 'pandas.core.frame.DataFrame'>
<class 'dict'>


{'rowData': array([  1,   4,   9,  16,  25,  36,  49,  64,  81, 100, 121, 169, 196,
        225]),
 '.TAU_application': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'ADD': array([201., 201., 201., 201., 201., 201., 201., 201., 201., 201., 201.,
        201., 201., 201.]),
 'ADI': array([201., 201., 201., 201., 201., 201., 201., 201., 201., 201., 201.,
        201., 201., 201.]),
 'ALLOC_SPACE': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'BTIO_CLEANUP': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'COMPUTE_BUFFER_SIZE': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'COMPUTE_RHS': array([202., 202., 202., 202., 202., 202., 202., 202., 202., 202., 202.,
        202., 202., 202.]),
 'COPY_FACES': array([202., 202., 202., 202., 202., 202., 202., 202., 202., 202., 202.,
        202., 202., 202.]),
 'ERROR_NORM': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'EXACT_RHS': array([1., 1., 1.,

In [36]:
### 引数に「return_dict_DataFrame()」の返り値をとる
### 返り値は行・列がモデル名・関数名で要素がMAPE値となっているDataFrame
def return_MapeTable_per_benchmark(dict_data :dict):

    # 変数：_names
    # 引数の辞書のプロセス数もしくはベンチマーククラスの文字列のリスト
    _names = list(dict_data.keys())
    _names.remove('rowData')
    # 変数：function_names
    # 引数の辞書の関数名の文字列のリスト
    function_names = dict_data['rowData']
    print(_names)
    print(function_names)


return_MapeTable_per_benchmark(dict_sample)


['.TAU_application', 'ADD', 'ADI', 'ALLOC_SPACE', 'BTIO_CLEANUP', 'COMPUTE_BUFFER_SIZE', 'COMPUTE_RHS', 'COPY_FACES', 'ERROR_NORM', 'EXACT_RHS', 'EXACT_SOLUTION', 'INITIALIZE', 'LHSABINIT', 'LHSINIT', 'MAKE_SET', 'MPBT', 'MPI_Allreduce()', 'MPI_Barrier()', 'MPI_Bcast()', 'MPI_Comm_dup()', 'MPI_Comm_rank()', 'MPI_Comm_size()', 'MPI_Finalize()', 'MPI_Init()', 'MPI_Reduce()', 'RHS_NORM', 'SETUP_BTIO', 'SETUP_MPI', 'SET_CLASS', 'SET_CONSTANTS', 'VERIFY', 'X_BACKSUBSTITUTE', 'X_SOLVE', 'X_SOLVE_CELL', 'Y_BACKSUBSTITUTE', 'Y_SOLVE', 'Y_SOLVE_CELL', 'Z_BACKSUBSTITUTE', 'Z_SOLVE', 'Z_SOLVE_CELL', 'MPI_Irecv()', 'MPI_Isend()', 'MPI_Wait()', 'MPI_Waitall()', 'X_RECEIVE_BACKSUB_INFO', 'X_RECEIVE_SOLVE_INFO', 'X_SEND_BACKSUB_INFO', 'X_SEND_SOLVE_INFO', 'X_UNPACK_BACKSUB_INFO', 'X_UNPACK_SOLVE_INFO', 'Y_RECEIVE_BACKSUB_INFO', 'Y_RECEIVE_SOLVE_INFO', 'Y_SEND_BACKSUB_INFO', 'Y_SEND_SOLVE_INFO', 'Y_UNPACK_BACKSUB_INFO', 'Y_UNPACK_SOLVE_INFO', 'Z_RECEIVE_BACKSUB_INFO', 'Z_RECEIVE_SOLVE_INFO', 'Z_SEND_B